# 问题背景
在金融业非常发达的今日，金融知识也不断的普及，而金融知识与数学知识、计算机知识密不可分。随着金融的不断发展，金融衍生工具也在走入越来越来多人的生活中。

期权是人们为了规避市场风险而创造出来的一种金融衍生工具，标的物常为股票或其他金融指标，给予持有者某种权利去做某件事，但期权持有者不一定必须行使权利。期权分为两种基本类型：看涨期权和看跌期权，又可以根据执行时间分为：不能提前行权的欧式期权和允许提前行权的美式期权，这些都属于常规期权。又在常规期权的基础上产生更为复杂的奇异期权，分为：障碍期权、亚式期权、打包期权和回溯期权等，奇异期权产生的原因许许多多，但归根结底都是因为通过奇异期权可以使交易商获得更高的利润。障碍期权是指当期权的价格高于或低于该障碍价格时，自动放弃期权，即期权价值变为0的某种奇异期权。

在《期权、期货及其他衍生产品》<sup>[1]</sup>的166-182中，介绍了常规期权中的是否提前行权下的期权头寸，即具体收益情况，同时也介绍了比较常见的期权标的物，以及决定期权价格的具体细节，以及一些期权认购与执行时的规则要求。这是这些常规期权最基本的知识，也是了解、研究的基础。

在《期权、期货及其他衍生产品》<sup>[1]</sup>的468-489中，介绍了各种类型的奇异期权，及其定价方法，同时也讨论了如何对某些奇异期权进行对冲。

理论和实践均表明，只要投资者合理地选择证券和相应衍生产品的比例，就可以进行套利，即获得无风险收益。这种组合的确定依赖于衍生产品的定价，所以如何对衍生产品的定价吸引了一批又一批的人进行研究。本文通过二叉树模型，总结前人的经验，利用Python面向对象的编程方式，对欧式、美式和障碍期权的数值定价进行研究与总结。

在《金融数量分析 基于Python编程》<sup>[2]</sup>的118-120中，介绍了期权二叉树定价模型，二叉树模型的基本原理是：假设变量运动只有向上和向下两个方向，且假设在整个考察期内，标的变量每次向上或向下的概率和幅度不变。将考察期分为若干阶段，根据标的变量的历史波动率模拟标的变量在整个考察期内所有可能的发展路径，并由后向前以倒推的形式走过所有结点，同时用贴现法得到在0时刻的价格。如果存在提前行权的问题(即美式期权的定价问题)，必须在二叉树的每个结点处检查在这一点行权是否比下一个结点上更有利，然后重复上述过程。

通过《二叉树与布莱克-斯科尔斯模型对欧式期权定价的比较分析》<sup>[3]</sup>中，作者通过以美股AAPL为标的资产，比较了B-S模型与二叉树模型在不同执行价格下的估值效果。作者发现模拟步数较少时，二叉树模型正确度比不上B-S模型，但当模拟步数大于2000步后，两者准确性相差不大。当执行价格处于平值附近时，二叉树模型对期权估值误差达到最大值，同时二叉树模型能有效对实值或虚值期权做出估值，且二叉树模型对欧式期权的估值效果较好，对美式看跌期权的估值接近真实值，对美式看涨期权的估值误差波动性较大。但是美式期权与普通欧式期权不同，在期权有效期内允许提前行权，不宜用B-S模型对美式期权进行定价，二叉树模型也仍是一个较优的模型。

通过上述期刊以及《A study of relevance of Black-Scholes model on option prices of Indian stock market》<sup>[4]</sup>，其作者都研究了波动性对估值结果的影响，数据证明二叉树模型在小波动$0.5\sigma$下，其估值稳定性优于B-S模型。在不稳定性低的时候，两个模型的估值有效性都较高。

在《金融数学》<sup>[5]</sup>的52-56页，简单介绍了如何对障碍期权进行定价。在障碍期权的二叉树模型定价上，可以通过设置障碍价格，将高于或低于障碍价格的期权价格置为0，从后往前计算，就可以比较容易的计算得到其定价，同时也较容易地编程实现。

# 问题表述

## 数学表述
欧式、美式与障碍期权的定价从数学上表述非常简洁：某种期权所对应的股票初始价格为$S_0$，期权的执行价为$K$，无风险利率为$r$，总时间期限为$T$，总期数为$N$(此处允许非整数时间步长的出现)，股票价格上涨和下跌的比率为$u,d$；对于障碍期权，还会有障碍价格$B$。在上述条件下求0时刻期权价格$V_0$。

## `IPO`方法

### 输入`Input`
在程序中输入的内容有：某种期权所对应的股票初始价格为$S_0$，期权的执行价为$K$，无风险利率为$r$，总时间期限为$T$，总期数为$N$，股票价格上涨和下跌的比率为$u,d$；对于障碍期权，还会有障碍价格$B$。同时，为了方便判断期权类型，还有参数$is\_put$是否为看跌期权，$is\_am$是否为美式期权，$is\_bar$是否为障碍期权，$is\_down$是否为向下期权，$is\_in$是否为敲入期权，障碍价格$B$。

### 处理`Process`
利用`Python`面向对象的方法以及二叉树模型的基本性质，通过定义父类与子类完成二叉树模型的具体编程，并通过数值方法计算欧式、美式与障碍期权的定价。从简单入手，定义基本期权类、欧式期权类、美式期权类(同时也可以进行欧式期权的定价)、障碍期权类(同时也可以进行欧式与美式期权的定价)，层层递进，简单易懂。

### 输出`Output`
程序输出的结果只有相应期权的定价，但是在执行过程中，我们可以记录下每一步的期权值，方便我们后续使用与研究分析。

# 问题解决

## 算法描述
在《Python金融数据分析》<sup>[6]</sup>中，作者介绍了如何通过二叉树模型，并结合Python面向对象编程的方法，对常见期权定价，以及使用有限差分法求解部分奇异期权的价格。虽然本文不采用有限差分法对障碍期权进行定价，但是这也为我们提供了新的思路，值得后续的研究，为方便说明，不妨利用Python的拓展包`pyreverse`(目前无法单独安装该包，但可以通过安装`pylint`达到同样的目的)，类图如下：

<img src=class_pic.png width=50% />

## 伪代码

### 欧式期权二叉树模型定价伪代码
<img src=eu_binarytree.png width=60% />

### 美式(含欧式)期权二叉树模型定价伪代码
<img src=am_eu_binarytree.png width=60% />

### 障碍期权二叉树模型定价伪代码
<img src=bar_binarytree.png width=60% />

# 代码实现

## 期权基本类`StockOption`

### 期权基本类`StockOption`的定义

In [1]:
# 定义期权基本类StockOption
import math

class StockOption(object):
    def __init__(self, S0, K, r, T, N, u = 1, d = 1, is_put = False, is_am = False, is_bar = False, is_down = False, is_in = False, B = None):
        """
        定义期权的基本信息：S0：初始股价，K：期权执行价，
        r：无风险利率，T：到期时间，N：期数，
        u/d：股价上涨/下跌的比率，is_put：是否看跌，
        is_am：是否美式，is_bar：是否障碍，
        is_down：是否向下，is_in：是否敲入，B：障碍价格
        """
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N)
        self.u, self.d = u, d
        self.is_call = not is_put
        self.is_european = not is_am
        self.is_bar = is_bar
        self.is_up = not is_down
        self.is_out = not is_in
        self.B = B
        # 初始化股票二叉树节点数值列表
        self.STs = []

    # 计算时间步长dt和贴现率df
    @property
    def dt(self):
        return self.T/float(self.N)

    @property
    def df(self):
        return math.exp(-self.r*self.dt)

### 期权基本类`StockOption`的举例应用
设某期权的参数$S_0 = 50, K = 52, r = 0.05$，若$T = 3, N = 12$，求时间步长$dt$与贴现率$df$。

In [2]:
options = StockOption(S0 = 50, K = 52, r = 0.05, T = 3, N = 12)
print('时间步长dt：', options.dt)
print('贴现率df：', options.df)

时间步长dt： 0.25
贴现率df： 0.9875778004938814


## 欧式期权二叉树类`BinomialEuropeanOption`

### `BinomialEuropeanOption`类的定义

In [3]:
# 定义欧式期权二叉树类
import math
import numpy as np

class BinomialEuropeanOption(StockOption):
    def setup_parameters(self):
        """
        定义二叉树的基本信息：
        M：N期的节点数，u/d：上涨/下跌的比率，
        qu/qd：上涨/下跌情况下的风险中性概率
        """
        self.M = self.N+1
        self.qu = (math.exp(self.r*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def init_stock_price_tree(self):
        # 计算股票二叉树的节点数值
        self.STs = np.zeros(self.M)
        for i in range(self.M):
            self.STs[i] = self.S0 * (self.u**(self.N-i)) * (self.d**i)

    def init_payoffs_tree(self):
        # 计算期权二叉树的节点数值
        if self.is_call:
            return np.maximum(0, self.STs-self.K)
        else:
            return np.maximum(0, self.K-self.STs)

    def traverse_tree(self, payoffs):
        # 反向计算期权二叉树的数值公式
        for i in range(self.N):
            payoffs = (payoffs[:-1]*self.qu + payoffs[1:]*self.qd)*self.df

        return payoffs

    def begin_tree_traversal(self):
        # 计算期权二叉树的数值
        payoffs = self.init_payoffs_tree()
        return self.traverse_tree(payoffs)

    def price(self):
        # 计算期权初始价格
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()

        # 返回期权最开始节点，即时刻0的价格
        return payoffs[0]

### `BinomialEuropeanOption`类的举例应用
假设参数为$S_0 = 50, K = 52, u = 1.2, d = 0.8$的欧式看跌期权，求$t=0$时刻其定价，其中$r=0.05,T=N=2$：

In [4]:
eu_option = BinomialEuropeanOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2, u = 1.2, d = 0.8, is_put = True)
print(eu_option.price())

4.1926542806038585


## 期权二叉树类`BinomialTreeOption`

### `BinomialTreeOption`类的定义

事实上，在美式期权类的定义过程中，只需稍加改动代码，即可同时实现欧式和美式期权的定价，因此可以通过定义`BinomialTreeOption`类，同时实现欧式和美式期权的定价。

In [5]:
# 定义期权二叉树类
import math
import numpy as np

class BinomialTreeOption(StockOption):
    def setup_parameters(self):
        """
        定义二叉树的基本信息：
        u/d：上涨/下跌的比率，
        qu/qd：上涨/下跌情况下的风险中性概率
        """
        self.qu = (math.exp(self.r*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def init_stock_price_tree(self):
        # 计算股票二叉树节点数值
        self.STs = [np.array([self.S0])]
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches*self.u, [prev_branches[-1]*self.d]))
            self.STs.append(st)

    def init_payoffs_tree(self):
        # 计算期权二叉树的节点数值
        if self.is_call:
            return np.maximum(0, self.STs[self.N]-self.K)
        else:
            return np.maximum(0, self.K-self.STs[self.N])

    def check_early_exercise(self, payoffs, node):
        # 对期权二叉树的节点数值判断是否提前行权
        if self.is_call:
            return np.maximum(payoffs, self.STs[node] - self.K)
        else:
            return np.maximum(payoffs, self.K - self.STs[node])

    def traverse_tree(self, payoffs):
        for i in reversed(range(self.N)):
            # 不提前行权时的价格(欧式)
            po = (payoffs[(self.N)-i-1][:-1]*self.qu + payoffs[(self.N)-i-1][1:]*self.qd)*self.df
            payoffs.append(po)

            # 提前行权时的价格(美式)
            if not self.is_european:
                po = self.check_early_exercise(payoffs[(self.N)-i], i)
                payoffs[(self.N)-i] = po

        return payoffs

    def begin_tree_traversal(self):
        # 计算期权二叉树的数值
        payoffs = [self.init_payoffs_tree()]
        return self.traverse_tree(payoffs)

    def price(self):
        # 计算期权初始价格
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        # 返回期权最开始节点，即时刻0的价格
        return payoffs[-1][0]

### `BinomialTreeOption`类的举例应用
假设参数为$S_0 = 50, K = 52, u = 1.2, d = 0.8$的看跌期权，其中$r=0.05,T=N=2$。若为欧式，$t=0$时刻其价格为多少，若为美式呢？

In [6]:
eu_option = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2, u = 1.2, d = 0.8, is_put = True)
am_option = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2, u = 1.2, d = 0.8, is_put = True, is_am = True)
print("欧式：", eu_option.price())
print("美式：", am_option.price())

欧式： 4.1926542806038585
美式： 5.089632474198373


## 障碍期权二叉树类`BinomialBarrierOption`

### `BinomialBarrierOption`类的定义

障碍期权的定价相比欧式与美式期权，多的一步是判断股票价格与障碍价格的大小关系，而且要根据障碍期权的具体分类，判别是障碍价格上方的价格有效还是下方的价格有效，根据《期权定价的数学模型和方法》<sup>[7]</sup>，障碍期权可以分为向上敲入期权(股票价格上涨到障碍价格时，期权开始生效)、向上敲出期权(股票价格上涨到障碍价格时，期权停止生效)、向下敲入期权(股票价格下跌到障碍价格时，期权开始生效)和向下敲出期权(股票价格下跌到障碍价格时，期权停止生效)。

只需在`BinomialTreeOption`类的基础上增加判断障碍价格的函数即可完成`BinomialBarrierOption`类的定义。

In [7]:
import math
import numpy as np

class BinomialBarrierOption(StockOption):
    def setup_parameters(self):
        """
        定义二叉树的基本信息：
        u/d：上涨/下跌的比率，
        qu/qd：上涨/下跌情况下的风险中性概率
        """
        self.qu = (math.exp(self.r*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def init_stock_price_tree(self):
        # 计算股票二叉树节点数值
        self.STs = [np.array([self.S0])]
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches*self.u, [prev_branches[-1]*self.d]))
            self.STs.append(st)

    def init_payoffs_tree(self):
        # 计算期权二叉树的节点数值
        if self.is_call:
            return np.maximum(0, self.STs[self.N]-self.K)
        else:
            return np.maximum(0, self.K-self.STs[self.N])

    def check_early_exercise(self, payoffs, node):
        # 对期权二叉树的节点数值判断是否提前行权
        if self.is_call:
            return np.maximum(payoffs, self.STs[node] - self.K)
        else:
            return np.maximum(payoffs, self.K - self.STs[node])

    def check_barrier_price(self, payoffs, node):
        for tf in zip(self.STs[node] >= self.B, list(range(node+1))):
            if tf[0] and self.is_up and self.is_out:
                payoffs[tf[1]] = 0
            elif (not tf[0]) and self.is_up and (not self.is_out):
                payoffs[tf[1]] = 0
            elif (not tf[0]) and (not self.is_up) and self.is_out:
                payoffs[tf[1]] = 0
            elif tf[0] and (not self.is_up) and (not self.is_out):
                payoffs[tf[1]] = 0
        return payoffs

    def traverse_tree(self, payoffs):
        for i in reversed(range(self.N)):
            if self.is_bar:
                po = self.check_barrier_price(payoffs[(self.N)-i-1], i+1)
                payoffs[(self.N)-i-1] = po

            # 不提前行权时的价格(欧式)
            po = (payoffs[(self.N)-i-1][:-1]*self.qu + payoffs[(self.N)-i-1][1:]*self.qd)*self.df
            payoffs.append(po)

            # 提前行权时的价格(美式)
            if not self.is_european:
                po = self.check_early_exercise(payoffs[(self.N)-i], i)
                payoffs[(self.N)-i] = po

        return payoffs

    def begin_tree_traversal(self):
        # 计算期权二叉树的数值
        payoffs = [self.init_payoffs_tree()]
        return self.traverse_tree(payoffs)

    def price(self):
        # 计算期权初始价格
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        # 返回期权最开始节点，即时刻0的价格
        return payoffs[-1][0]

### `BinomialBarrierOption`类的举例应用
假设参数为$S_0 = 50, K = 52, u = 1.2, d = 0.8$的欧式看跌期权，其中$r=0.05,T=N=3$。若期权为向下敲出期权，障碍价格为$B = 35$，求$t=0$时刻其价格。

In [8]:
bar_option = BinomialBarrierOption(S0 = 50, K = 52, r = 0.05, T = 3, N = 3, u = 1.2, d = 0.8, is_put = True, is_bar = True, is_down = True, is_in = False, B = 35)
print(bar_option.price())

2.0331903766709143


# 数值示例

## 例1
假设参数为$S_0 = 50, K = 52, u = 1.2, d = 0.8$的看涨期权，其中$r=0.05,T=2$。若
1. $N = 2$；
2. $N = 20$；
3. $N = 200$；
4. $N = 2000$；

若期权为欧式，$t=0$时刻其价格为多少，若为美式呢？

In [9]:
eu_option_1 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2, u = 1.2, d = 0.8, is_put = False)
eu_option_2 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 20, u = 1.2, d = 0.8, is_put = False)
eu_option_3 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 200, u = 1.2, d = 0.8, is_put = False)
eu_option_4 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2000, u = 1.2, d = 0.8, is_put = False)
print('欧式看涨期权：', eu_option_1.price(), eu_option_2.price(), eu_option_3.price(), eu_option_4.price())

am_option_1 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2, u = 1.2, d = 0.8, is_put = False, is_am = True)
am_option_2 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 20, u = 1.2, d = 0.8, is_put = False, is_am = True)
am_option_3 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 200, u = 1.2, d = 0.8, is_put = False, is_am = True)
am_option_4 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2000, u = 1.2, d = 0.8, is_put = False, is_am = True)
print('美式看涨期权：', am_option_1.price(), am_option_2.price(), am_option_3.price(), am_option_4.price())

欧式看涨期权： 7.141108542733969 18.29971524033906 42.52279270504153 49.999680293038
美式看涨期权： 7.141108542733969 18.29971524033906 42.52279270504153 49.99968029303846


## 例2
假设参数为$S_0 = 50, K = 52, u = 1.2, d = 0.8$的看跌期权，其中$r=0.05,T=2$。若
1. $N = 2$；
2. $N = 20$；
3. $N = 200$；
4. $N = 2000$；

若期权为欧式，$t=0$时刻其价格为多少，若为美式呢？

In [10]:
eu_option_1 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2, u = 1.2, d = 0.8, is_put = True)
eu_option_2 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 20, u = 1.2, d = 0.8, is_put = True)
eu_option_3 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 200, u = 1.2, d = 0.8, is_put = True)
eu_option_4 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2000, u = 1.2, d = 0.8, is_put = True)
print('欧式看跌期权：', eu_option_1.price(), eu_option_2.price(), eu_option_3.price(), eu_option_4.price())

am_option_1 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2, u = 1.2, d = 0.8, is_put = True, is_am = True)
am_option_2 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 20, u = 1.2, d = 0.8, is_put = True, is_am = True)
am_option_3 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 200, u = 1.2, d = 0.8, is_put = True, is_am = True)
am_option_4 = BinomialTreeOption(S0 = 50, K = 52, r = 0.05, T = 2, N = 2000, u = 1.2, d = 0.8, is_put = True, is_am = True)
print('美式看跌期权：', am_option_1.price(), am_option_2.price(), am_option_3.price(), am_option_4.price())

欧式看跌期权： 4.1926542806038585 15.351260978209066 39.57433844291228 47.051226030905525
美式看跌期权： 5.089632474198373 16.221716750389685 41.18821951270311 51.11875701132149


通过例1和例2，我们会发现美式看跌期权的价格高于或等于欧式看跌期权。对于这个现象，同样也有一些文献进行了介绍和证明，在《金融工程|衍生品|期权价格的上下限》<sup>[8]</sup>中，通过反证法和组合比较法证明了对于看涨期权来说，期权价格$V \leq S_0$，且对于理性投资人来说，欧式看涨期权的价格等于美式看涨期权的价格。但是对于看跌期权来说，欧式期权的价格上限为：$P_{eu} \leq Ke^{-rT}$，美式期权的价格上限为：$P_{am} \leq K$，因此，美式看跌期权的价格大于等于对应的欧式看跌期权。

## 例3
1. 利用下图的股价二叉树，并设置向下敲出的障碍为跌破65美元，$X = 50$美元，$r = 0.06$，求$t = 0$时刻看涨期权的价格。

![example3-1](example3-1.png)

2. 利用下图的股价二叉树，并设置向上敲出的障碍为115美元，$X = 85$美元，$r = 0.06$，求$t = 0$时刻看涨期权的价格。

![example3-2](example3-2.png)

In [11]:
bar_option_1 = BinomialBarrierOption(S0 = 70, K = 50, r = 0.06, T = 3, N = 3, u = 1.25, d = 0.8, is_bar = True, is_down = True, is_in = False, B = 65)
bar_option_2 = BinomialBarrierOption(S0 = 100, K = 85, r = 0.06, T = 3, N = 3, u = 1.1, d = 0.9, is_bar = True, is_down = False, is_in = False, B = 115)
print(bar_option_1.price(), bar_option_2.price())

23.137416107948216 5.291152881491191


## 例4
1. 针对例3，若期权不是障碍期权，而是欧式期权，计算$t = 0$时刻看涨期权的价格。
2. 针对例3中1的数据，若
   1. $B=40$；
   2. $B=50$；
   3. $B=60$；
   4. $B=70$；

   计算障碍期权与对应欧式看涨期权的价差。

In [12]:
reg_option_11 = BinomialBarrierOption(S0 = 70, K = 50, r = 0.06, T = 3, N = 3, u = 1.25, d = 0.8)
reg_option_12 = BinomialBarrierOption(S0 = 100, K = 85, r = 0.06, T = 3, N = 3, u = 1.1, d = 0.9)
print('对应欧式期权：', reg_option_11.price(), reg_option_12.price())
bar_option_21 = BinomialBarrierOption(S0 = 70, K = 50, r = 0.06, T = 3, N = 3, u = 1.25, d = 0.8, is_bar = True, is_down = True, is_in = False, B = 40)
bar_option_22 = BinomialBarrierOption(S0 = 70, K = 50, r = 0.06, T = 3, N = 3, u = 1.25, d = 0.8, is_bar = True, is_down = True, is_in = False, B = 50)
bar_option_23 = BinomialBarrierOption(S0 = 70, K = 50, r = 0.06, T = 3, N = 3, u = 1.25, d = 0.8, is_bar = True, is_down = True, is_in = False, B = 60)
bar_option_24 = BinomialBarrierOption(S0 = 70, K = 50, r = 0.06, T = 3, N = 3, u = 1.25, d = 0.8, is_bar = True, is_down = True, is_in = False, B = 75)
print('价差：', reg_option_11.price() - bar_option_21.price(), reg_option_11.price() - bar_option_22.price(), reg_option_11.price() - bar_option_23.price(), reg_option_11.price() - bar_option_24.price())


对应欧式期权： 29.10117563591397 29.072252766419204
价差： 0.0 0.5098489358197789 5.963759527965756 10.397972248472175


我们发现障碍期权的价格低于对应欧式期权的价格，且障碍价格越高，价差越多。在《基于二叉树方法的障碍期权与标准期权价差分析模型》<sup>[9]</sup>中，作者运用二叉树方法建立了障碍期权与标准期权之间的价差分析模型，也发现障碍期权的价值低于对应的标准期权。并且，对于下降敲出看涨期权，若障碍值或行权价越高、有效期越长，标的物的价格越低，则障碍期权与标准期权的价差越大。

# 总结
二叉树一种常用的期权定价方法，相比BSM模型，这个方法适用范围更广，可以为欧式期权、美式期权等一些其他品种定价。该方法是保持波动率不变的条件下，将价格路径做简化，根据简化的路径做分析和计算。

本文首先介绍使用二叉树为期权定价的原理，然后给出为欧式期权和美式期权定价的程序。使用二叉树进行期权定价，主要包含两步：
1. 从左向右生成股价二叉树；
2. 根据生成的股价二叉树，从右向左计算期权价值。

在完成代码的过程中，先建立期权基本类`StockOption`，再到欧式期权类`BinomialEuropeanOption`，欧式期权不会提前行权，情况较为简单。首先计算叶子节点的期权价值，其次向前加权平均并折现，得到前一层节点的期权价值，最后重复上述步骤至$t=0$时刻。

美式期权可能提前行权，要在欧式期权的基础上加一步是否提前行权的判断，经过增加判断是否提前行权，建立美式(含欧式)期权类`BinomialTreeOption`。首先计算叶子节点的期权价值，其次向前加权平均并折现，得到前一层节点的期权价值。然后判断在该节点是否提前行权，若提前行权的话，将提前行权的期权价值更新为本节点的期权价值。最后不断重复上述过程至$t=0$时刻。

障碍期权存在期权停止生效的问题，所以要判断障碍价格与股票价格之间的大小关系，建立障碍期权类`BinomialBarrierOption`。首先计算叶子节点的期权价值，其次向前加权平均并折现，得到前一层节点的期权价值。然后判断在该节点所对应的股票价格节点与障碍价格的大小关系，以更新障碍期权节点的数值，后续步骤与欧式、美式期权的定价类似，最后不断重复上述过程至$t=0$时刻。并且`BinomialBarrierOption`也可以用于对常规的欧式与美式期权定价。整个代码呈现自下而上的思路，符合面向对象编程的思路。

当然，当二叉树模型扩展到$n$步后，其计算的方法仍然是相同的，从后往前依次计算出每个节点的期权价格，并乘以对应的风险中性概率，直到求出$t=0$时刻的期权价格，无论是上述哪种期权，都可以通过类似的方法最后算得$t=0$时刻的期权价格。

同时，通过代码下方的简单示例与数值示例，我们也可以得到一些关于期权的有趣的结论：
1. 欧式看涨期权的价格等于美式看涨期权的价格，且上限为$K$；
2. 欧式看跌期权的价格上限为$Ke^{-rT}$，美式看跌期权的价格上限为$K$，美式看跌期权的价格大于等于对应的欧式看跌期权；
3. 障碍期权的价值低于对应的标准期权；
4. 对于下降敲出看涨期权，若障碍值或行权价越高、有效期越长，标的物的价格越低，则障碍期权与标准期权的价差越大。

检验发现，当$n$达到比较大时，二叉树模型所确定的期权价格趋于一个平稳的值(如下图所示)，这也在一定程度上说明了二叉树模型的合理性和实用性。背后的原理应该是二项分布的极限是正态分布，即步数足够多时，沿二项分布路径得到的股票价格，是趋向于正态分布的<sup>[10]</sup>。在《期权定价中二叉树模型的极限情况》<sup>[11]</sup>中，作者证明了当二叉树模型中的时间间隔趋向于零的时候，二叉树模型就演化成了B-S模型，即二叉树模型的极限形式就是B-S模型，这也再次证明了二叉树模型的有效性。

<img src=binarylimit.jpg width=50% />


# 参考文献
[1] [加]John C. Hull. 期权、期货及其他衍生产品[M], 王勇, 索吾林译. 第十版. 北京:机械工业出版社, 2018:166-182, 468-489.

[2] 郑志勇, 怀伟城, 王玮珩. 金融数量分析 基于Python编程[M]. 第一版. 北京:北京航空航天大学出版社, 2018:118-120.

[3] 何一若. 二叉树与布莱克-斯科尔斯模型对欧式期权定价的比较分析[J]. 内蒙古民族大学学报(社会科学版), 2021, 47(5):88-94.

[4] Srivastava A., Shastri M. A study of relevance of Black-Scholes model on option prices of Indian stock market[J]. International Journal Governance and Financial Intermediation, 2018, 1(1):82-104.

[5] [美]Joseph Stampfli, Victor Goodman. 金融数学[M], 蔡明超译. 第一版. 北京:机械工业出版社, 2004:52-56.

[6] [新]马伟明. Python金融数据分析[M], 张永翼, 霍达, 张彤译. 第一版. 北京:机械工业出版社, 2018:53-91.

[7] 姜礼尚. 期权定价的数学模型和方法[M]. 第一版. 北京:高等教育出版社, 2003:247-255.

[8] 金融工程|衍生品|期权价格的上下限[EB/OL]. (2020-02-01)[2022-06-30]. https://www.fxtjz.com/upper-and-lower-bound-of-option-prices/#European_call_American_call-2.

[9] 胡文伟, 李湛. 基于二叉树方法的障碍期权与标准期权价差分析模型[J]. 上海交通大学学报,2012,46(5):825-831.

[10] 期权的二叉树定价模型[EB/OL]. (2022-05-24)[2022-07-01]. https://zhuanlan.zhihu.com/p/255135694.

[11] 曲军恒, 张占英, 赵春茹. 期权定价中二叉树模型的极限情况[J]. 许昌学院学报, 2007, 26(5):1-5.
